# **Inference Proyek Fundamental Deep Learning**

## **Import Semua Packages/Library yang Digunakan**

In [1]:
# Import Library untuk Inference
import pickle
import numpy as np
import tensorflow as tf
import re
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import _VectorizerMixin

# Nonaktifkan peringatan warning
warnings.filterwarnings('ignore')

# Konfigurasi TensorFlow (opsional)
tf.get_logger().setLevel('ERROR')

## **Load Model untuk Inference**

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import os

folder_path = '/content/drive/MyDrive/Instagram Review'

if os.path.exists(folder_path):
    print(f"Files in {folder_path}:")
    for filename in os.listdir(folder_path):
        print(filename)
else:
    print(f"Folder not found at {folder_path}")

Files in /content/drive/MyDrive/Instagram Review:
svm_tfidf_8020_model (2).pkl
tfidf_vectorizer (3).pkl
tokenizer.pkl
svm_tfidf_6040_model (2).pkl
svm_bow_8020_model.pkl
svm_ngram_8020_model (1).pkl
lstm_tfidf_702010_model (1).h5
rf_tfidf_7030_model (2).pkl
rf_tfidf_8020_model (2).pkl
data_bersih_berlabel (3).csv


In [14]:
# Load Data from Google Drive
ulasan = pd.read_csv('/content/drive/MyDrive/Instagram Review/data_bersih_berlabel (3).csv')
display(ulasan.head())

,content,score,word_count,clean_content,casefold_content,tokens,stemmed_tokens,tokens_negation,tokens_slang,stopword_removal,sentiment_label,sentiment_score
0,Mantap 🔥,5,2,Mantap 🔥,mantap 🔥,"['mantap', '🔥']","['mantap', '']","['mantap', '']","['mantap', '']","['mantap', '']",positif,1
1,"Gatau kenapa, dari kemarin tuh aku gabisa buat...",1,30,Gatau kenapa dari kemarin tuh aku gabisa buat ...,gatau kenapa dari kemarin tuh aku gabisa buat ...,"['gatau', 'kenapa', 'dari', 'kemarin', 'tuh', ...","['gatau', 'kenapa', 'dari', 'kemarin', 'tuh', ...","['gatau', 'kenapa', 'dari', 'kemarin', 'tuh', ...","['gatau', 'kenapa', 'dari', 'kemarin', 'tuh', ...","['gatau', 'kemarin', 'tuh', 'tidak bisa', 'pos...",positif,1
2,Dukun aku buat menjangkau bintang direels,5,6,Dukun aku buat menjangkau bintang direels,dukun aku buat menjangkau bintang direels,"['dukun', 'aku', 'buat', 'menjangkau', 'bintan...","['dukun', 'aku', 'buat', 'jangkau', 'bintang',...","['dukun', 'aku', 'buat', 'jangkau', 'bintang',...","['dukun', 'aku', 'buat', 'jangkau', 'bintang',...","['dukun', 'jangkau', 'bintang', 'direels']",negatif,-1
3,ada apakah selalu for close pas mau membuka ap...,1,11,ada apakah selalu for close pas mau membuka ap...,ada apakah selalu for close pas mau membuka ap...,"['ada', 'apakah', 'selalu', 'for', 'close', 'p...","['ada', 'apakah', 'selalu', 'for', 'close', 'p...","['ada', 'apakah', 'selalu', 'for', 'close', 'p...","['ada', 'apakah', 'selalu', 'for', 'tutup', 'p...","['tutup', 'pas', 'buka', 'aplikasi', 'instagram']",netral,0
4,Good,5,1,Good,good,['good'],['good'],['good'],['bagus'],['bagus'],positif,1


In [15]:
# Ekstrak Data untuk Klasifikasi
texts = ulasan['stopword_removal'].tolist()
labels = ulasan['sentiment_label'].tolist()

## **Load Model**

In [16]:
# Memuat Vectorizer
with open('/content/drive/MyDrive/Instagram Review/tfidf_vectorizer (3).pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('/content/drive/MyDrive/Instagram Review/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Memuat SVM Model
with open('/content/drive/MyDrive/Instagram Review/svm_tfidf_8020_model (2).pkl', 'rb') as f:
    svm_tfidf_8020 = pickle.load(f)

with open('/content/drive/MyDrive/Instagram Review/svm_tfidf_6040_model (2).pkl', 'rb') as f:
    svm_tfidf_6040 = pickle.load(f)

with open('/content/drive/MyDrive/Instagram Review/svm_ngram_8020_model (1).pkl', 'rb') as f:
    svm_ngram_8020 = pickle.load(f)

with open('/content/drive/MyDrive/Instagram Review/svm_bow_8020_model.pkl', 'rb') as f:
    svm_bow_8020 = pickle.load(f)

# Memuat Random Forest Model
with open('/content/drive/MyDrive/Instagram Review/rf_tfidf_8020_model (2).pkl', 'rb') as f:
    rf_tfidf_8020 = pickle.load(f)

with open('/content/drive/MyDrive/Instagram Review/rf_tfidf_7030_model (2).pkl', 'rb') as f:
    rf_tfidf_7030 = pickle.load(f)

# Memuat LSTM Model
lstm_tfidf_8020 = tf.keras.models.load_model('/content/drive/MyDrive/Instagram Review/lstm_tfidf_702010_model (1).h5')

## **Preprocessing**

Membuat Data Baru Untuk Prediksi

In [17]:
#Data Baru
new_texts = [
    "suka banget ini aplikasi bagus!",  # sentimen positif
    "tidak bisa masuk gagal terus",  # sentimen negatif
    "kualitasnya biasa saja."  # netral
]

Preprocessing Teks

In [18]:
# Preprocessing Teks SVM dan RF
new_texts_cleaned = []
for text in new_texts:
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join(text.split())
    new_texts_cleaned.append(text)

In [19]:
# Preprocessing untuk Model LSTM:
new_sequences = tokenizer.texts_to_sequences(new_texts_cleaned)
new_padded = pad_sequences(new_sequences, maxlen=100)

In [20]:
# Transform data baru dengan TF-IDF
data_new_tfidf = tfidf_vectorizer.transform(new_texts_cleaned)

In [21]:
ngram_vectorizer_new = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=2000,
)

ngram_vectorizer_new.fit(texts)

TfidfVectorizer(max_features=2000, ngram_range=(1, 2))

In [22]:
# Transform data baru dengan N-Gram
data_new_ngram = ngram_vectorizer_new.transform(new_texts_cleaned)
data_new_ngram_dense = data_new_ngram.toarray()

In [35]:
bow_vectorizer_new = TfidfVectorizer(
    max_features=8510,
)

bow_vectorizer_new.fit(texts)

TfidfVectorizer(max_features=8510)

In [36]:
data_new_bow = bow_vectorizer_new.transform(new_texts_cleaned)
data_new_bow_dense = data_new_bow.toarray()

## **Prediksi**

### **N-Gram dan BoW**

In [37]:
svm_ngram_8020_pred = svm_ngram_8020.predict(data_new_ngram_dense)

In [38]:
# Now predict with the SVM BoW model
svm_bow_8020_pred = svm_bow_8020.predict(data_new_bow_dense)

In [39]:
print(svm_ngram_8020.n_features_in_)

2000


In [41]:
print(data_new_tfidf.shape[1])

9907


### **TF-IDF**

In [42]:
# SVM Prediksi
svm_tfidf_8020_pred = svm_tfidf_8020.predict(data_new_tfidf)
svm_tfidf_6040_pred = svm_tfidf_6040.predict(data_new_tfidf)

In [43]:
# Random Forest Prediksi
rf_tfidf_8020_pred = rf_tfidf_8020.predict(data_new_tfidf)
rf_tfidf_7030_pred = rf_tfidf_7030.predict(data_new_tfidf)

In [44]:
# LSTM Prediksi
lstm_preds_classes = np.argmax(lstm_tfidf_8020.predict(new_padded), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 779ms/step


## **Hasil**

In [45]:
# Mapping label numerik ke label string
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Tampilkan hasil prediksi
print("\nSentiment Prediction Results:")
for i, text in enumerate(new_texts):
    print(f"Text: {text}")
    print(f"LSTM Prediction: {label_map[lstm_preds_classes[i]]}")
    print(f"SVM N-Gram Prediction: {svm_ngram_8020_pred[i]}")
    print(f"SVM BoW Prediction: {svm_bow_8020_pred[i]}")
    print(f"SVM TF-IDF (8020) Prediction: {label_map[svm_tfidf_8020_pred[i]]}")
    print(f"SVM TF-IDF (6040) Prediction: {label_map[svm_tfidf_6040_pred[i]]}")
    print(f"RF TF-IDF (8020) Prediction: {label_map[rf_tfidf_8020_pred[i]]}")
    print(f"RF TF-IDF (7030) Prediction: {label_map[rf_tfidf_7030_pred[i]]}")
    print("-" * 50)


Sentiment Prediction Results:
Text: suka banget ini aplikasi bagus!
LSTM Prediction: neutral
SVM N-Gram Prediction: negatif
SVM BoW Prediction: netral
SVM TF-IDF (8020) Prediction: positive
SVM TF-IDF (6040) Prediction: positive
RF TF-IDF (8020) Prediction: positive
RF TF-IDF (7030) Prediction: positive
--------------------------------------------------
Text: tidak bisa masuk gagal terus
LSTM Prediction: neutral
SVM N-Gram Prediction: negatif
SVM BoW Prediction: netral
SVM TF-IDF (8020) Prediction: negative
SVM TF-IDF (6040) Prediction: negative
RF TF-IDF (8020) Prediction: negative
RF TF-IDF (7030) Prediction: negative
--------------------------------------------------
Text: kualitasnya biasa saja.
LSTM Prediction: neutral
SVM N-Gram Prediction: netral
SVM BoW Prediction: netral
SVM TF-IDF (8020) Prediction: neutral
SVM TF-IDF (6040) Prediction: neutral
RF TF-IDF (8020) Prediction: neutral
RF TF-IDF (7030) Prediction: neutral
--------------------------------------------------


Notebook ini mendemonstrasikan proses melakukan inferensi sentimen pada data teks menggunakan beberapa model machine learning yang berbeda. Berikut adalah ringkasan bagian-bagian utamanya:


Impor Library : Notebook dimulai dengan mengimpor library yang diperlukan untuk manipulasi data (pandas, numpy), pemrosesan teks (re), model machine learning (tensorflow, sklearn), dan penanganan file (pickle, os).


Memuat Data : Notebook ini me-mount Google Drive untuk mengakses file yang tersimpan di sana, kemudian memuat file CSV bernama data_bersih_berlabel (3).csv ke dalam pandas DataFrame. DataFrame ini berisi data teks dan label sentimen yang terkait.


Ekstraksi Data  : Data teks (kolom stopword_removal) dan label sentimen (kolom sentiment_label) diekstraksi dari DataFrame.


Memuat Model  : Notebook ini memuat beberapa model machine learning yang sudah dilatih sebelumnya dan vectorizer/tokenizer terkait dari file .pkl dan .h5 yang tersimpan di Google Drive. Model-model ini mencakup SVM dengan fitur TF-IDF, N-Gram, dan BoW, Random Forest dengan fitur TF-IDF, dan model LSTM.


Preprocessing : Data teks baru untuk inferensi didefinisikan. Data baru ini menjalani langkah-langkah preprocessing yang serupa dengan apa yang kemungkinan diterapkan pada data pelatihan, termasuk pembersihan, mengubah ke huruf kecil, dan tokenisasi. Data kemudian ditransformasikan menggunakan vectorizer TF-IDF, N-Gram, dan BoW yang dimuat, serta dipadatkan untuk model LSTM.


Prediksi  : Data teks baru yang sudah dipreprocessing dimasukkan ke dalam setiap model yang dimuat untuk mendapatkan prediksi sentimen.


Hasil : Label sentimen yang diprediksi dari setiap model untuk setiap teks baru dicetak dan ditampilkan, bersama dengan teks aslinya.